## 1. Подключение к Spark

In [2]:
from pyspark.sql import SparkSession

# Имя хоста Spark Master в docker
SPARK_MASTER_HOST = "spark-master"
# Порт Spark Master
SPARK_MASTER_PORT = "7077"
# Память выделенная для Spark Worker в настройках docker-compose.yml
SPARK_WORKER_MEMORY = "512m"
# Название сессии (любое)
SPARK_SESSION = "pyspark-notebook"

# Создаем сессию
spark = (
    SparkSession.builder.appName(SPARK_SESSION)
    .master(f"spark://{SPARK_MASTER_HOST}:{SPARK_MASTER_PORT}")
    .config("spark.executor.memory", SPARK_WORKER_MEMORY)
    .getOrCreate()
)

## Чтение в Spark из Jupyter workspace

Прочитать файл в Spark можно из рабочей директории Jupyter Lab

In [ ]:
# Директория с данными в workspace Jupyter
BASE_DIR = './data/'
# Название файла
FILENAME = 'test.csv'
# Полное имя файла
file = f"{BASE_DIR}{FILENAME}"

# Чтение файла
spark_df = spark.read.csv(file, header=True)

## Запись в Jupyter workspace

In [ ]:
FILENAME = 'test.csv'

write_jupyter = (
    spark_df.write.option("header", True)
    # .partitionBy("PublishYear")
    .mode("overwrite")
    .parquet(f'{BASE_DIR}{FILENAME}')
)

## Чтение в Spark из Hadoop

In [ ]:
# Имя Hadoop Namenode в docker
HADOOP_HOST = 'hadoop-namenode'
# Hadoop Namenode port
HADOOP_PORT = '9000'
# Название файла
FILENAME = 'test.parquet'

spark_df = spark.read.parquet(f"hdfs://{HADOOP_HOST}:{HADOOP_PORT}/{FILENAME}")

## Запись в Hadoop

In [ ]:
FILENAME = 'test.parquet'

write_hadoop = (
    spark_df.write.option("header", True)
    # .partitionBy("PublishYear")
    .mode("overwrite")
    .parquet(f"hdfs://{HADOOP_HOST}:{HADOOP_PORT}/{FILENAME}")
)

## Преобразовать данные исходного датасета в parquet объединяя все таблицы.

получаем список файлов

In [22]:
import os

file_list = [f"{BASE_DIR}{file}" for file in os.listdir(BASE_DIR) if 'book' in file]
file_list

['./data/book1-100k.csv',
 './data/book1400k-1500k.csv',
 './data/book1100k-1200k.csv',
 './data/book1800k-1900k.csv',
 './data/book1700k-1800k.csv',
 './data/book500k-600k.csv',
 './data/book1300k-1400k.csv',
 './data/book1000k-1100k.csv',
 './data/book700k-800k.csv',
 './data/book1500k-1600k.csv',
 './data/book1900k-2000k.csv',
 './data/book900k-1000k.csv',
 './data/book4000k-5000k.csv',
 './data/book400k-500k.csv',
 './data/book800k-900k.csv',
 './data/book1200k-1300k.csv',
 './data/book3000k-4000k.csv',
 './data/book600k-700k.csv',
 './data/book100k-200k.csv',
 './data/book300k-400k.csv',
 './data/book1600k-1700k.csv',
 './data/book2000k-3000k.csv',
 './data/book200k-300k.csv']

Берем схему из первого файла

In [47]:
from pyspark.sql.types import StructType, StructField, StringType

with open(file_list[0]) as file:
    headers = file.readline().rstrip().split(',')

schema = StructType([StructField(header, StringType(), True) for header in headers])

emp_RDD = spark.sparkContext.emptyRDD()

spark_df = spark.createDataFrame(data = emp_RDD, schema=schema)

In [48]:
spark_df.show()

+---+----+-----------+-----------+-----------+---------------+------------+----------+---------+--------------+-----------+--------+-------+------+-----------+-----------+----+-----------+
| Id|Name|RatingDist1|pagesNumber|RatingDist4|RatingDistTotal|PublishMonth|PublishDay|Publisher|CountsOfReview|PublishYear|Language|Authors|Rating|RatingDist2|RatingDist5|ISBN|RatingDist3|
+---+----+-----------+-----------+-----------+---------------+------------+----------+---------+--------------+-----------+--------+-------+------+-----------+-----------+----+-----------+
+---+----+-----------+-----------+-----------+---------------+------------+----------+---------+--------------+-----------+--------+-------+------+-----------+-----------+----+-----------+



In [54]:
for file in file_list:
    spark_data = spark.read.csv(file, header=True)
    # spark_df = spark_df[headers]
    spark_df = spark_df.unionByName(spark_data, allowMissingColumns=True)

In [56]:
(
    spark_df.write.option("header", True)
    # .partitionBy("PublishYear")
    .mode("overwrite")
    .parquet(f"hdfs://{HADOOP_HOST}:{HADOOP_PORT}/books.parquet")
)

In [63]:
spark_df = spark_df.dropDuplicates()

In [64]:
spark_df.filter(spark_df['Id'] == '400000').show(vertical=True)

-RECORD 0-------------------------------------
 Id                    | 400000               
 Name                  | The Gigli Concert    
 RatingDist1           | 1:2                  
 pagesNumber           | 96                   
 RatingDist4           | 4:10                 
 RatingDistTotal       | total:27             
 PublishMonth          | 16                   
 PublishDay            | 9                    
 Publisher             | Bloomsbury Methue... 
 CountsOfReview        | 2                    
 PublishYear           | 1991                 
 Language              | null                 
 Authors               | Tom    Murphy        
 Rating                | 3.3                  
 RatingDist2           | 2:4                  
 RatingDist5           | 5:3                  
 ISBN                  | 0413659305           
 RatingDist3           | 3:8                  
 Description           | null                 
 Count of text reviews | null                 



In [65]:
spark_df.count()

1873851

In [141]:
df_load = spark.read.parquet(f"hdfs://{HADOOP_HOST}:{HADOOP_PORT}/books.parquet")

In [142]:
df_load.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- RatingDist1: string (nullable = true)
 |-- pagesNumber: string (nullable = true)
 |-- RatingDist4: string (nullable = true)
 |-- RatingDistTotal: string (nullable = true)
 |-- PublishMonth: string (nullable = true)
 |-- PublishDay: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- CountsOfReview: string (nullable = true)
 |-- PublishYear: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- Authors: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- RatingDist2: string (nullable = true)
 |-- RatingDist5: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- RatingDist3: string (nullable = true)



In [143]:
df_load.tail(10)

[Row(Id='299975', Name="Visual C# 2005: A Developer's Notebook: A Developer's Notebook", RatingDist1='1:1', pagesNumber='240', RatingDist4='4:5', RatingDistTotal='total:16', PublishMonth='2', PublishDay='5', Publisher="O'Reilly Media", CountsOfReview='1', PublishYear='2005', Language=None, Authors='Jesse Liberty', Rating='3.44', RatingDist2='2:2', RatingDist5='5:3', ISBN='059600799X', RatingDist3='3:5'),
 Row(Id='299976', Name='Silent Joe', RatingDist1='1:33', pagesNumber='341', RatingDist4='4:725', RatingDistTotal='total:1780', PublishMonth='25', PublishDay='4', Publisher='Hyperion Books', CountsOfReview='124', PublishYear='2001', Language='eng', Authors='T. Jefferson Parker', Rating='3.85', RatingDist2='2:94', RatingDist5='5:476', ISBN='0786867280', RatingDist3='3:452'),
 Row(Id='299977', Name='A Silent Sorrow: Pregnancy Loss-- Guidance and Support for You and Your Family', RatingDist1='1:0', pagesNumber='299', RatingDist4='4:24', RatingDistTotal='total:62', PublishMonth='20', Publis

In [145]:
df_load.filter(df_load['Id'] == '1').show(truncate=False)

+---+------------------------------------------------------------+-----------+------------------+-----------+---------------+------------+----------+----------------------------------+--------------+-----------+--------+--------------------+----------------------------------------------------+-----------+-----------+----+--------------------+
|Id |Name                                                        |RatingDist1|pagesNumber       |RatingDist4|RatingDistTotal|PublishMonth|PublishDay|Publisher                         |CountsOfReview|PublishYear|Language|Authors             |Rating                                              |RatingDist2|RatingDist5|ISBN|RatingDist3         |
+---+------------------------------------------------------------+-----------+------------------+-----------+---------------+------------+----------+----------------------------------+--------------+-----------+--------+--------------------+----------------------------------------------------+-----------+----

In [91]:
schema

StructType([StructField('Id', StringType(), True), StructField('Name', StringType(), True), StructField('RatingDist1', StringType(), True), StructField('pagesNumber', StringType(), True), StructField('RatingDist4', StringType(), True), StructField('RatingDistTotal', StringType(), True), StructField('PublishMonth', StringType(), True), StructField('PublishDay', StringType(), True), StructField('Publisher', StringType(), True), StructField('CountsOfReview', StringType(), True), StructField('PublishYear', StringType(), True), StructField('Language', StringType(), True), StructField('Authors', StringType(), True), StructField('Rating', StringType(), True), StructField('RatingDist2', StringType(), True), StructField('RatingDist5', StringType(), True), StructField('ISBN', StringType(), True), StructField('RatingDist3', StringType(), True)])

In [120]:
spark_df = spark.read.csv('./data/book200k-300k.csv', header=True, inferSchema=True)

In [122]:
spark_df = spark_df[headers]

In [124]:
spark_df.head()

Row(Id='200000', Name='Meet My Staff', RatingDist1='1:4', pagesNumber='40', RatingDist4='4:7', RatingDistTotal='total:27', PublishMonth='11', PublishDay=9, Publisher='HarperCollins', CountsOfReview='5', PublishYear='1998', Language=None, Authors='Patricia Marx', Rating='3.52', RatingDist2='2:3', RatingDist5='5:9', ISBN='0060274840', RatingDist3='3:4')

In [123]:
spark_df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- RatingDist1: string (nullable = true)
 |-- pagesNumber: string (nullable = true)
 |-- RatingDist4: string (nullable = true)
 |-- RatingDistTotal: string (nullable = true)
 |-- PublishMonth: string (nullable = true)
 |-- PublishDay: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- CountsOfReview: string (nullable = true)
 |-- PublishYear: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- Authors: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- RatingDist2: string (nullable = true)
 |-- RatingDist5: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- RatingDist3: string (nullable = true)



In [84]:
spark_df1 = spark.read.csv('./data/book1-100k.csv', header=True)

In [85]:
spark_df2 = spark.read.csv('./data/book100k-200k.csv', header=True)

In [86]:
df3 = spark_df1.unionByName(spark_df2)

In [89]:
df3.filter(df3['Id'] == '2').show(truncate=False)

+---+------------------------------------------------------------+-----------+-----------+-----------+---------------+------------+----------+---------------+--------------+-----------+--------+------------+------+-----------+-----------+----------+-----------+
|Id |Name                                                        |RatingDist1|pagesNumber|RatingDist4|RatingDistTotal|PublishMonth|PublishDay|Publisher      |CountsOfReview|PublishYear|Language|Authors     |Rating|RatingDist2|RatingDist5|ISBN      |RatingDist3|
+---+------------------------------------------------------------+-----------+-----------+-----------+---------------+------------+----------+---------------+--------------+-----------+--------+------------+------+-----------+-----------+----------+-----------+
|2  |Harry Potter and the Order of the Phoenix (Harry Potter, #5)|1:12455    |870        |4:604283   |total:2358637  |1           |9         |Scholastic Inc.|29770         |2004       |eng     |J.K. Rowling|4.5   |

In [46]:
spark_df2.printSchema()

root
 |-- pagesNumber: string (nullable = true)
 |-- Authors: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- RatingDistTotal: string (nullable = true)
 |-- RatingDist5: string (nullable = true)
 |-- RatingDist3: string (nullable = true)
 |-- CountsOfReview: string (nullable = true)
 |-- PublishDay: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- RatingDist4: string (nullable = true)
 |-- PublishMonth: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- PublishYear: string (nullable = true)
 |-- RatingDist1: string (nullable = true)
 |-- RatingDist2: string (nullable = true)
 |-- Name: string (nullable = true)



In [76]:
spark_df.filter(spark_df['Id'] == '2').show(truncate=False)

+---+------------------------------------------------------------+------------------------------------------------------+-----------+-----------+---------------+------------+----------+---------------+--------------+-----------+--------+------------+------+-----------+-----------+----------+----------------------------------------------------------------------------------+
|Id |Name                                                        |RatingDist1                                           |pagesNumber|RatingDist4|RatingDistTotal|PublishMonth|PublishDay|Publisher      |CountsOfReview|PublishYear|Language|Authors     |Rating|RatingDist2|RatingDist5|ISBN      |RatingDist3                                                                       |
+---+------------------------------------------------------------+------------------------------------------------------+-----------+-----------+---------------+------------+----------+---------------+--------------+-----------+--------+-----------

In [9]:
df.show()

+-------+--------------------+--------------------+----------+------+-----------+------------+----------+--------------------+-----------+-----------+-----------+-----------+-----------+---------------+--------------+--------+-----------+--------------------+
|     Id|                Name|             Authors|      ISBN|Rating|PublishYear|PublishMonth|PublishDay|           Publisher|RatingDist5|RatingDist4|RatingDist3|RatingDist2|RatingDist1|RatingDistTotal|CountsOfReview|Language|PagesNumber|         Description|
+-------+--------------------+--------------------+----------+------+-----------+------------+----------+--------------------+-----------+-----------+-----------+-----------+-----------+---------------+--------------+--------+-----------+--------------------+
|4000000|      Top Management|     Bernard  Taylor|0582446058|   0.0|       1973|           1|         1|Longman Publishin...|        5:0|        4:0|        3:0|        2:0|        1:0|        total:0|             0|   

In [3]:
spark.stop()

## 2. Connection to Spark Cluster

To connect to the Spark cluster, create a SparkSession object with the following params:

+ **appName:** application name displayed at the [Spark Master Web UI](http://localhost:8080/);
+ **master:** Spark Master URL, same used by Spark Workers;
+ **spark.executor.memory:** must be less than or equals to docker compose SPARK_WORKER_MEMORY config.

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

## 3. Load and Store Data
We will now load data from a local CSV and store it to Hadoop partitioned by column.
Afterward you can access Hadoop UI to explore the saved parquet files.
Access Hadoop UI on 'http://localhost:9870' (Utilities -> Browse the files system )

In [ ]:
import pandas
from pyspark.sql.types import *
from pyspark.sql import functions as F
import os
import time    
epochNow = int(time.time())

In [ ]:
#Iterate over all files until we find the sales file and then creates a Pandas dataframe.
for path, subdirs, files in os.walk('./data/'):
    for name in files:
        if "salesRecord" in name:
            csvName = name
            csvPath = os.path.join(path, name)
            print("Loading data from csv {}".format(csvPath))
            salesDfPandas = pandas.read_csv(csvPath)

In [ ]:
#Create PySpark DataFrame from Pandas
salesDfSpark=spark.createDataFrame(salesDfPandas)

In [ ]:
#Remove spaces in column names
salesDfSpark = salesDfSpark.select([F.col(col).alias(col.replace(' ', '_')) for col in salesDfSpark.columns])
print("Sales Dataframe created with schema : ")
salesDfSpark.printSchema()

In [ ]:
# Write Dataframe into HDFS
# Repartition it by "Country" column before storing as parquet files in Hadoop
salesDfSpark.write.option("header",True) \
        .partitionBy("Country") \
        .mode("overwrite") \
        .parquet("hdfs://hadoop-namenode:9000/sales/{}_{}.parquet".format(csvName,epochNow))
print("Sales Dataframe stored in Hadoop.")

In [ ]:
# Read from HDFS to confirm it was successfully stored
df_load = spark.read.parquet("hdfs://hadoop-namenode:9000/sales/{}_{}.parquet".format(csvName,epochNow))
print("Sales Dataframe read from Hadoop : ")
df_load.show()